In [1]:
""" This module implements Berstein-Vairani algorithm """
from sympy.physics.quantum.qapply import qapply
from sympy.physics.quantum.qubit import Qubit, matrix_to_qubit
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.gate import HadamardGate
from sympy.physics.quantum.represent import represent
from sympy.physics.quantum.qubit import Qubit,  measure_partial_oneshot
from sympy import preorder_traversal

from oracle import oracle
from util.util import hn
import numpy as np


def fbv(x, s):
    """
    Calculate dot product with XOR instead of SUM
    """
    r = 0
    for a, b in zip(x, s):
        r ^= a * b
    return r


def berstein_vazirani(s, n):
    """
    Run Berstein-Vazirani algorithm and return |x>*|y XOR f(x)>
    :param int s: bit string string
    :param int n: string length

    The problem:
    We have a function f {0,1}^n -> {0,1}
    f(x) = s dot x
    s is a n-bit string
    s dot x = s_n-1x_n-1 + ... + s1x1 + s0x0
    We want to find s = s_n-1...s1s0

    The Solution:
                         +-----------------+
      |x>  |0>-/n--H*n---|                 |--/n--H*n---M |x>
                         |       U_f       |
      |y>  |1>---H-------|                 |------------  |y XOR f(x)>
                         +-----------------+

    """
    x = qapply(hn(n) * Qubit('0' * n))
    print(f"|x>: {x}")
    y = qapply(HadamardGate(0) * Qubit(1))
    print(f"|y>: {y}")

    # calculate the Tensor Product of the inputs
    xy = TensorProduct(x, y)
    xy = matrix_to_qubit(represent(xy))
    print(f"|xy>: {xy}")

    # apply oracle
    r = oracle(x, y, fbv, s)

    # apply H*n on top Qubits
    r = qapply(hn(n, start=1) * r)
    return r


In [2]:
""" This module implements Berstein-Vairani algorithm """
from sympy.physics.quantum.qapply import qapply
from sympy.physics.quantum.qubit import Qubit, matrix_to_qubit, measure_partial_oneshot
from sympy.physics.quantum import TensorProduct
from sympy.physics.quantum.gate import HadamardGate
from sympy.physics.quantum.represent import represent
from sympy import preorder_traversal

from oracle import oracle
from util.util import hn

In [3]:
def fbv(x, s):
    """
    Calculate dot product with XOR instead of SUM
    """
    r = 0
    for a, b in zip(x, s):
        r ^= a * b
    return r

In [4]:
def berstein_vazirani(s, n):
    """
    Run Berstein-Vazirani algorithm and return |x>*|y XOR f(x)>
    :param int s: bit string string
    :param int n: string length

    The problem:
    We have a function f {0,1}^n -> {0,1}
    f(x) = s dot x
    s is a n-bit string
    s dot x = s_n-1x_n-1 + ... + s1x1 + s0x0
    We want to find s = s_n-1...s1s0

    The Solution:
                         +-----------------+
      |x>  |0>-/n--H*n---|                 |--/n--H*n---M |x>
                         |       U_f       |
      |y>  |1>---H-------|                 |------------  |y XOR f(x)>
                         +-----------------+

    """
    x = qapply(hn(n) * Qubit('0' * n))
    print(f"|x>: {x}")
    y = qapply(HadamardGate(0) * Qubit(1))
    print(f"|y>: {y}")

    # calculate the Tensor Product of the inputs
    xy = TensorProduct(x, y)
    xy = matrix_to_qubit(represent(xy))
    print(f"|xy>: {xy}")

    # apply oracle
    r = oracle(x, y, fbv, s)

    # apply H*n on top Qubits
    r = qapply(hn(n, start=1) * r)
    return r

In [5]:
def test_0():
    print()
    size = 4
    s = np.random.choice([0, 1], size=(size,))
    print(f'\ns: {s}')

    r = berstein_vazirani(s, size)
    print(f'Result: {r}')

    m = measure_partial_oneshot(r, range(size))
    print(f'Measure: {m}')
    s_found = None
    # skip y part of the measure
    for arg in preorder_traversal(m):
        if isinstance(arg, Qubit):
            s_found = Qubit(*arg.qubit_values[0:size])

    assert Qubit(*s) == s_found

test_0()



s: [1 1 1 1]
|x>: |0000>/4 + |0001>/4 + |0010>/4 + |0011>/4 + |0100>/4 + |0101>/4 + |0110>/4 + |0111>/4 + |1000>/4 + |1001>/4 + |1010>/4 + |1011>/4 + |1100>/4 + |1101>/4 + |1110>/4 + |1111>/4
|y>: sqrt(2)*|0>/2 - sqrt(2)*|1>/2
|xy>: sqrt(2)*|00000>/8 - sqrt(2)*|00001>/8 + sqrt(2)*|00010>/8 - sqrt(2)*|00011>/8 + sqrt(2)*|00100>/8 - sqrt(2)*|00101>/8 + sqrt(2)*|00110>/8 - sqrt(2)*|00111>/8 + sqrt(2)*|01000>/8 - sqrt(2)*|01001>/8 + sqrt(2)*|01010>/8 - sqrt(2)*|01011>/8 + sqrt(2)*|01100>/8 - sqrt(2)*|01101>/8 + sqrt(2)*|01110>/8 - sqrt(2)*|01111>/8 + sqrt(2)*|10000>/8 - sqrt(2)*|10001>/8 + sqrt(2)*|10010>/8 - sqrt(2)*|10011>/8 + sqrt(2)*|10100>/8 - sqrt(2)*|10101>/8 + sqrt(2)*|10110>/8 - sqrt(2)*|10111>/8 + sqrt(2)*|11000>/8 - sqrt(2)*|11001>/8 + sqrt(2)*|11010>/8 - sqrt(2)*|11011>/8 + sqrt(2)*|11100>/8 - sqrt(2)*|11101>/8 + sqrt(2)*|11110>/8 - sqrt(2)*|11111>/8
Result: sqrt(2)*|11110>/2 - sqrt(2)*|11111>/2
Measure: -|11111>
